# Homework07

Exercises to practice pandas, data analysis and regression

## Goals

- Understand the effects of pre-processing data
- Get familiar with the ML flow: encode -> normalize -> train -> evaluate
- Understand the difference between regression and classification tasks
- Build intuition for different regression models

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor

from data_utils import object_from_json_url
from data_utils import regression_error

### Load Dataset

Let's load up the full [ANSUR](https://www.openlab.psu.edu/ansur2/) dataset that we looked at briefly in [Week 02](https://github.com/DM-GY-9103-2024F-H/WK02).

This is the dataset that has anthropometric information about U.S. Army personnel.

#### WARNING

Like we mentioned in class, this dataset is being used for these exercises due to the level of detail in the dataset and the rigorous process that was used in collecting the data.

This is a very specific dataset and should not be used to draw general conclusions about people, bodies, or anything else that is not related to the distribution of physical features of U.S. Army personnel.

In [ ]:
# Load Dataset
ANSUR_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/main/datasets/json/ansur.json"
ansur_data = object_from_json_url(ANSUR_FILE)

# Look at first 2 records
ansur_data[:2]

#### Nested data

This is that *nested* dataset from Week 02.

# 🤔

Let's load it into a `DataFrame` to see what happens.

In [ ]:
# Read into DataFrame
ansur_df = pd.DataFrame.from_records(ansur_data)
ansur_df.head()


# 😓🙄

That didn't work too well. We ended up with objects in our columns.

Luckily, our `DataFrame` library has a function called [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) that can help.

In [ ]:
# Read into DataFrame
ansur_df = pd.json_normalize(ansur_data)
ansur_df.head()

Much better. `DataFrames` are magic.

#### Data Exploration

Before we start creating models, let's do a little bit of data analysis and get a feeling for the shapes, distributions and relationships of our data.

1. Print `min`, `max` and `average` values for all of the features.
2. Print `covariance` tables for `age`, `ear.length` and `head.circumference`.
3. Plot `age`, `ear.length` and `head.circumference` versus the $1$ *feature* that is most correlated to each of them.

Don't forget to *encode* and *normalize* the data.

In [ ]:
# Work on Data Exploration here

### Encode non-numerical features
ansur_encoded_df = ansur_df.copy()

non_numeric_cols = ansur_encoded_df.select_dtypes(include=['object']).columns
print("Non-numeric columns:", list(non_numeric_cols))

if len(non_numeric_cols) > 0:
    encoder = OrdinalEncoder().set_output(transform="pandas")
    ansur_encoded_df[non_numeric_cols] = encoder.fit_transform(ansur_encoded_df[non_numeric_cols])
else:
    print("No categorical features to encode.")


ansur_num_df = ansur_encoded_df.select_dtypes(include=['number'])

## 1. Print min, max, avg

for feat in ansur_num_df.columns:
    print(f"{feat}")
    print(f"\tmin: {ansur_num_df[feat].min():.3f}")
    print(f"\tmax: {ansur_num_df[feat].max():.3f}")
    print(f"\tavg: {ansur_num_df[feat].mean():.3f}")

### Normalize all data

scaler = StandardScaler().set_output(transform="pandas")
ansur_scaled_df = scaler.fit_transform(ansur_num_df)
display(ansur_scaled_df.head())

## 2. Print Covariances

features_of_interest = ["age", "ear.length", "head.circumference"]
for feat in features_of_interest:
    print(f"\nCovariance matrix for {feat}:")
    display(ansur_scaled_df.cov()[[feat]].sort_values(by=feat, ascending=False).head(10))

## 3. Plot features most correlated to age, ear length and head circumference

for feat in features_of_interest:
    corr_series = ansur_scaled_df.corr()[feat].drop(labels=[feat])
    top_feat = corr_series.abs().idxmax()
    print(f"Feature most correlated with {feat}: {top_feat} (corr={corr_series[top_feat]:.3f})")


    plt.scatter(ansur_scaled_df[top_feat], ansur_scaled_df[feat],
                alpha=0.3, marker='o')
    plt.xlabel(top_feat)
    plt.ylabel(feat)
    plt.title(f"{feat} vs {top_feat}")
    plt.show()


### Interpretation

<span style="color:hotpink;">
Does anything stand out about these graphs? Or the correlations?<br>
Are correlations symmetric? Does the feature most correlated to ear length also have ear length as its most correlated pair?
</span>

<span style="color:hotpink;">EDIT THIS CELL WITH ANSWER</span>
The first graph, age vs ear length, shows a wide spread of points without any strong visible trend. The dots form a dense vertical cluster, meaning that ear length varies across individuals but not systematically with age. There’s no clear upward or downward slope. People of different ages have roughly similar ear lengths. This could reflect that age isn’t a determining factor for ear size once adulthood is reached or that the dataset’s age range doesn’t capture childhood growth where such changes might occur. Overall, it suggests a very weak or no linear correlation between the two.

The second graph, ear length vs weight, shows a clearer diagonal pattern. As weight increases, ear length also tends to increase slightly. The relationship is moderately positive and more consistent than with age, meaning heavier individuals often have somewhat longer ears in this dataset.

The third graph, head circumference vs head height, shows the strongest and most linear relationship of all three. The points form a tight diagonal band, suggesting a strong positive correlation. As head height increases, head circumference increases almost proportionally. This is expected since both features relate to head size.

These feature pairs were selected based on their highest covariance values from the table above. Each represents the most correlated relationship for its respective variable. Regarding the main question, correlations themselves are symmetric by definition. If ear length is correlated with weight, then weight is equally correlated with ear length. However, the ranking of strongest correlations is not always symmetric. While ear length’s top correlation might be with weight, weight’s top correlation might instead be with height or another size-related feature. This explains why correlation symmetry doesn’t necessarily imply mirrored feature rankings.


### Regression

Now, we want to create a regression model to predict `head.circumference` from the data.

From our [Week 07](https://github.com/PSAM-5020-2025F-A/WK07) notebook, we can create a regression model by following these steps:

1. Load dataset (done! 🎉)
2. Encode label features as numbers (done! ⚡️)
3. Normalize the data (done! 🍾)
4. Separate the outcome variable and the input features
5. Create a regression model using all features
6. Run model on training data and measure error
7. Plot predictions and interpret results
8. Run model on test data, measure error, plot predictions, interpret results

In [ ]:
# Work on Regression Model here

## Separate outcome variable and input features

outcome = ansur_num_df["head.circumference"]
features = ansur_scaled_df.drop(columns=["head.circumference"])

## Create a regression model
head_model = LinearRegression().fit(features, outcome)

## Measure error on training data
predicted_head = head_model.predict(features)
regression_error(outcome, predicted_head)

print("Training error:")
print(regression_error(outcome, predicted_head))

## Plot predictions and interpret results
head_original = outcome
plt.plot(sorted(head_original), marker='o', linestyle='', alpha=0.3)
plt.plot(sorted(predicted_head), color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("head.circumference")
plt.show()


In [ ]:
## Load Test Data
ANSUR_TEST_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/main/datasets/json/ansur-test.json"

ansur_test_data = object_from_json_url(ANSUR_TEST_FILE)
ansur_test_df = pd.json_normalize(ansur_test_data)

ansur_test_encoded_df = ansur_test_df.copy()

g_vals = encoder.transform(ansur_test_df[["gender"]].values)
ansur_test_encoded_df[["gender"]] = g_vals

ansur_test_scaled_df = scaler.transform(ansur_test_encoded_df)

In [ ]:
## Run model on test data

from sklearn.model_selection import train_test_split

features_train, features_test, outcome_train, outcome_test = train_test_split(
    features, outcome, test_size=0.2, random_state=42)


head_model = LinearRegression().fit(features_train, outcome_train)


## Measure error on test data

predicted_test = head_model.predict(features_test)
regression_error(outcome_test, predicted_test)

## Plot predictions and interpret results
plt.plot(sorted(outcome_test), marker='o', linestyle='', alpha=0.3)
plt.plot(sorted(predicted_test), color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("head.circumference")
plt.title("Test Data: Actual vs Predicted Head Circumference")
plt.show()


### Interpretation

<span style="color:hotpink;">
How well does your model perform?<br>
How could you improve it?<br>
Are there ranges of circumferences that don't get predicted well?
</span>

<span style="color:hotpink;">EDIT THIS CELL WITH ANSWER</span>
My model performs quite well overall. The training and test results both show very similar curves, which means the regression model has successfully captured the main linear relationships in the ANSUR dataset. The predicted values follow the same general pattern as the actual head circumference values, only with a smoother line. This smoothness makes sense because linear regression fits an average trend through the data rather than trying to match every small fluctuation. The fact that the test results look almost identical to the training results also shows that the model generalizes well and isn’t overfitting the data.

When I look closely at the plots, the model performs best across the middle range of head circumferences, where the majority of data points lie. However, the predictions become less accurate toward the extremes. For very small head circumferences, the model tends to overestimate and for very large ones, it slightly underestimates. This happens because there are fewer samples at the tails of the dataset and a simple linear model cannot bend to capture any non-linear relationships that might exist there. In other words, it pulls the extreme values toward the mean.

To improve the model, I could introduce non-linear features using polynomial terms so that it can better capture curvature at the extremes. Adding squared terms or interactions between key body measurements (like height, weight and head length) would allow the model to account for more complex relationships. 
